### this notebook is a copy of train.py 

In [ ]:
from rl.callbacks import ModelIntervalCheckpoint, FileLogger
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Concatenate
from gym import spaces
import numpy as np
from PIL import Image
from configparser import ConfigParser
import os
from os.path import join, pardir, exists

from gym_airsim.airsim_car_env import AirSimCarEnv

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

class AirSimCarProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        # We could perform this processing step in `process_observation`. In this case, however,
        # we would need to store a `float32` array instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

config = ConfigParser()
config.read('config.ini')
num_actions = int(config['car_agent']['actions'])
                    
WINDOW_LENGTH = 4
INPUT_SHAPE = (84, 84)

env = AirSimCarEnv()
np.random.seed(123)

# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_actions))
model.add(Activation('linear'))
print(model.summary())


def build_callbacks(env_name):
    log_dir = 'logs'
    if not exists(log_dir):
        os.makedirs(log_dir)
    
    checkpoint_weights_filename = join(log_dir, 'dqn_' + env_name + '_weights_{step}.h5f')
    log_filename = join(log_dir,'dqn_{}_log.json'.format(env_name))
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=25000)]
    callbacks += [FileLogger(log_filename, interval=100)]
    return callbacks

memory = SequentialMemory(limit=50000, window_length=WINDOW_LENGTH)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),  attr='eps', value_max=1., 
                              value_min=.1, value_test=.05, nb_steps=1000000)
processor = AirSimCarProcessor()

dqn = DQNAgent(model=model, nb_actions=num_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.0001), metrics=['mae'])

callbacks = build_callbacks('AirSimCarRL')

dqn.fit(env, nb_steps=2000000,
        visualize=False,
        verbose=2,
        callbacks=callbacks)

Using TensorFlow backend.
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\sit

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

['WayPoint0', 'WayPoint1', 'WayPoint2', 'WayPoint3', 'WayPoint4', 'WayPoint5', 'WayPoint6', 'WayPoint7', 'WayPoint8', 'WayPoint9']



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_____________________________________________________

    3383/2000000: episode: 17, duration: 9.572s, episode steps: 190, steps per second: 20, episode reward: 142.600, mean reward: 0.751 [-1.000, 1.000], mean action: 2.968 [0.000, 6.000], mean observation: 172.108 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    3565/2000000: episode: 18, duration: 8.958s, episode steps: 182, steps per second: 20, episode reward: 76.300, mean reward: 0.419 [-1.000, 0.500], mean action: 2.951 [0.000, 6.000], mean observation: 173.011 [23.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    3748/2000000: episode: 19, duration: 8.958s, episode steps: 183, steps per second: 20, episode reward: 116.300, mean reward: 0.636 [-1.000, 1.000], mean action: 3.131 [0.000, 6.000], mean observation: 171.915 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    3925/2000000: episode: 20, duration: 8.607s, episode steps: 177, steps per second: 21, episode reward: 80.400, mean rewar

    8833/2000000: episode: 46, duration: 7.686s, episode steps: 180, steps per second: 23, episode reward: 73.300, mean reward: 0.407 [-1.000, 0.500], mean action: 2.833 [0.000, 6.000], mean observation: 172.125 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    9013/2000000: episode: 47, duration: 7.582s, episode steps: 180, steps per second: 24, episode reward: 55.700, mean reward: 0.309 [-1.000, 0.500], mean action: 2.883 [0.000, 6.000], mean observation: 172.735 [21.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    9218/2000000: episode: 48, duration: 9.074s, episode steps: 205, steps per second: 23, episode reward: 78.600, mean reward: 0.383 [-1.000, 0.500], mean action: 2.971 [0.000, 6.000], mean observation: 172.466 [23.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
    9443/2000000: episode: 49, duration: 10.508s, episode steps: 225, steps per second: 21, episode reward: 137.600, mean rewar

   14798/2000000: episode: 75, duration: 9.678s, episode steps: 210, steps per second: 22, episode reward: 113.400, mean reward: 0.540 [-1.000, 1.000], mean action: 3.043 [0.000, 6.000], mean observation: 171.495 [23.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   15028/2000000: episode: 76, duration: 10.852s, episode steps: 230, steps per second: 21, episode reward: 143.700, mean reward: 0.625 [-1.000, 1.000], mean action: 3.122 [0.000, 6.000], mean observation: 172.358 [23.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   15198/2000000: episode: 77, duration: 7.170s, episode steps: 170, steps per second: 24, episode reward: 58.300, mean reward: 0.343 [-1.000, 0.500], mean action: 3.076 [0.000, 6.000], mean observation: 171.806 [24.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   15393/2000000: episode: 78, duration: 8.357s, episode steps: 195, steps per second: 23, episode reward: 76.800, mean rewa

...

  278405/2000000: episode: 1442, duration: 11.649s, episode steps: 290, steps per second: 25, episode reward: 149.600, mean reward: 0.516 [-1.000, 1.000], mean action: 3.317 [0.000, 6.000], mean observation: 173.444 [24.000, 255.000], loss: 0.047524, mean_absolute_error: 9.632207, mean_q: 11.248237, mean_eps: 0.749566
  278640/2000000: episode: 1443, duration: 9.384s, episode steps: 235, steps per second: 25, episode reward: 163.500, mean reward: 0.696 [-1.000, 1.000], mean action: 3.077 [0.000, 6.000], mean observation: 171.690 [24.000, 255.000], loss: 0.051694, mean_absolute_error: 9.864854, mean_q: 11.515477, mean_eps: 0.749330
  278891/2000000: episode: 1444, duration: 10.028s, episode steps: 251, steps per second: 25, episode reward: 154.300, mean reward: 0.615 [-1.000, 1.000], mean action: 2.980 [0.000, 6.000], mean observation: 172.253 [24.000, 255.000], loss: 0.055295, mean_absolute_error: 9.790292, mean_q: 11.430258, mean_eps: 0.749112
  279153/2000000: episode: 1445, duration

  284874/2000000: episode: 1468, duration: 9.403s, episode steps: 248, steps per second: 26, episode reward: 103.700, mean reward: 0.418 [-1.000, 0.500], mean action: 2.956 [0.000, 6.000], mean observation: 172.802 [24.000, 255.000], loss: 0.049505, mean_absolute_error: 10.613106, mean_q: 12.386958, mean_eps: 0.743725
  285137/2000000: episode: 1469, duration: 10.386s, episode steps: 263, steps per second: 25, episode reward: 177.200, mean reward: 0.674 [-1.000, 1.000], mean action: 3.251 [0.000, 6.000], mean observation: 172.605 [24.000, 255.000], loss: 0.044881, mean_absolute_error: 10.236789, mean_q: 11.945847, mean_eps: 0.743495
  285396/2000000: episode: 1470, duration: 9.992s, episode steps: 259, steps per second: 26, episode reward: 148.500, mean reward: 0.573 [-1.000, 1.000], mean action: 3.286 [0.000, 6.000], mean observation: 172.368 [24.000, 255.000], loss: 0.050077, mean_absolute_error: 10.151369, mean_q: 11.856105, mean_eps: 0.743261
  285666/2000000: episode: 1471, durati

  291453/2000000: episode: 1494, duration: 9.866s, episode steps: 252, steps per second: 26, episode reward: 146.900, mean reward: 0.583 [-1.000, 1.000], mean action: 2.790 [0.000, 6.000], mean observation: 171.008 [24.000, 255.000], loss: 0.053724, mean_absolute_error: 10.761686, mean_q: 12.586861, mean_eps: 0.737805
  291702/2000000: episode: 1495, duration: 9.651s, episode steps: 249, steps per second: 26, episode reward: 158.400, mean reward: 0.636 [-1.000, 1.000], mean action: 3.004 [0.000, 6.000], mean observation: 170.780 [23.000, 255.000], loss: 0.049286, mean_absolute_error: 10.653147, mean_q: 12.435465, mean_eps: 0.737580
  291969/2000000: episode: 1496, duration: 10.508s, episode steps: 267, steps per second: 25, episode reward: 117.200, mean reward: 0.439 [-1.000, 0.500], mean action: 3.277 [0.000, 6.000], mean observation: 172.779 [24.000, 255.000], loss: 0.059521, mean_absolute_error: 10.768606, mean_q: 12.578316, mean_eps: 0.737348
  292238/2000000: episode: 1497, durati

  297876/2000000: episode: 1520, duration: 9.686s, episode steps: 238, steps per second: 25, episode reward: 168.100, mean reward: 0.706 [-1.000, 1.000], mean action: 3.160 [0.000, 6.000], mean observation: 171.743 [24.000, 255.000], loss: 0.048342, mean_absolute_error: 10.574033, mean_q: 12.344408, mean_eps: 0.732020
  298109/2000000: episode: 1521, duration: 9.328s, episode steps: 233, steps per second: 25, episode reward: 176.100, mean reward: 0.756 [-1.000, 1.000], mean action: 2.987 [0.000, 6.000], mean observation: 171.875 [24.000, 255.000], loss: 0.055163, mean_absolute_error: 10.877304, mean_q: 12.717941, mean_eps: 0.731807
  298326/2000000: episode: 1522, duration: 7.948s, episode steps: 217, steps per second: 27, episode reward: 89.800, mean reward: 0.414 [-1.000, 0.500], mean action: 2.903 [0.000, 6.000], mean observation: 172.226 [23.000, 255.000], loss: 0.053259, mean_absolute_error: 10.980014, mean_q: 12.811782, mean_eps: 0.731604
  298559/2000000: episode: 1523, duration

  304325/2000000: episode: 1546, duration: 8.803s, episode steps: 234, steps per second: 27, episode reward: 145.900, mean reward: 0.624 [-1.000, 1.000], mean action: 3.218 [0.000, 6.000], mean observation: 171.702 [23.000, 255.000], loss: 0.071124, mean_absolute_error: 11.523122, mean_q: 13.484852, mean_eps: 0.726213
  304576/2000000: episode: 1547, duration: 9.897s, episode steps: 251, steps per second: 25, episode reward: 180.000, mean reward: 0.717 [-1.000, 1.000], mean action: 3.295 [0.000, 6.000], mean observation: 172.020 [24.000, 255.000], loss: 0.061618, mean_absolute_error: 11.338889, mean_q: 13.251302, mean_eps: 0.725995
  304812/2000000: episode: 1548, duration: 9.477s, episode steps: 236, steps per second: 25, episode reward: 174.400, mean reward: 0.739 [-1.000, 1.000], mean action: 3.123 [0.000, 6.000], mean observation: 171.412 [23.000, 255.000], loss: 0.055700, mean_absolute_error: 11.109517, mean_q: 12.976601, mean_eps: 0.725777
  305092/2000000: episode: 1549, duratio

### testing

In [ ]:
log_dir = 'logs'
if not exists(log_dir):
    os.makedirs(log_dir)
    
weights_filename = join(log_dir,'dqn_{}_weights.h5f'.format('AirSimCarRL'))
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=True)